<a href="https://colab.research.google.com/github/gregorywmorris/MLZoom2022/blob/main/capstone-1/LendingClub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Auto-save / Auto-reload
%autosave 1

%load_ext autoreload
%autoreload 2

Autosaving every 1 seconds


Field descriptions: https://www.kaggle.com/code/faressayah/lending-club-loan-defaulters-prediction

# Environment set-up and data import

## PIP

In [41]:
!pip install dask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [90]:
!pip install -U dataprep

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Envrionment

In [88]:
import numpy as np
import pandas as pd
import dataprep
from dataprep.eda import create_report, plot
import sys

# visualization
from IPython.display import display
import seaborn as sns
import matplotlib.pyplot as plt


In [44]:
sys.setrecursionlimit(10000)

In [45]:
data1 = "https://raw.githubusercontent.com/gregorywmorris/MLZoom2022/main/capstone-1/lendingclub-1.csv"

In [46]:
data2 = "https://raw.githubusercontent.com/gregorywmorris/MLZoom2022/main/capstone-1/lendingclub-2.csv"

In [47]:
!wget $data1 -O lendingclub-1.csv

--2022-12-17 23:45:54--  https://raw.githubusercontent.com/gregorywmorris/MLZoom2022/main/capstone-1/lendingclub-1.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13161065 (13M) [text/plain]
Saving to: ‘lendingclub-1.csv’

lendingclub-1.csv   100%[===================>]  12.55M  --.-KB/s    in 0.1s    

2022-12-17 23:45:54 (91.0 MB/s) - ‘lendingclub-1.csv’ saved [13161065/13161065]



In [48]:
!!wget $data2 -O lendingclub-2.csv

['--2022-12-17 23:45:54--  https://raw.githubusercontent.com/gregorywmorris/MLZoom2022/main/capstone-1/lendingclub-2.csv',
 'Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...',
 'Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.',
 'HTTP request sent, awaiting response... 200 OK',
 'Length: 15921810 (15M) [text/plain]',
 'Saving to: ‘lendingclub-2.csv’',
 '',
 '',
 'lendingclub-2.csv     0%[                    ]       0  --.-KB/s               ',
 'lendingclub-2.csv   100%[===================>]  15.18M  91.9MB/s    in 0.2s    ',
 '',
 '2022-12-17 23:45:54 (91.9 MB/s) - ‘lendingclub-2.csv’ saved [15921810/15921810]',
 '']

In [49]:
part1 = pd.read_csv("lendingclub-1.csv")

In [50]:
part2 = pd.read_csv("lendingclub-2.csv")

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,47) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [51]:
bothParts = [part1,part2]
df = pd.concat(bothParts)

# EDA

## Data Cleaning / Preprocessing

In [52]:
df.isnull().sum()

id                       42537
member_id                42538
loan_amnt                    3
funded_amnt                  3
funded_amnt_inv              3
                         ...  
settlement_status        42378
settlement_date          42378
settlement_amount        42378
settlement_percentage    42378
settlement_term          42378
Length: 144, dtype: int64

In [53]:
df.shape

(42538, 144)

In [54]:
# drop columns that have no values
for col in df.columns:
  if df[col].isna().sum() >= 42537:
    df.drop(col,inplace=True,axis=1)

In [55]:
# confirm dropped columns
df.shape

(42538, 62)

In [56]:
# check dtypes
df.dtypes

loan_amnt                float64
funded_amnt              float64
funded_amnt_inv          float64
term                      object
int_rate                  object
                          ...   
settlement_status         object
settlement_date           object
settlement_amount        float64
settlement_percentage    float64
settlement_term          float64
Length: 62, dtype: object

In [84]:
for obj in df.columns[df.dtypes == 'O']:
    print(df[obj].value_counts())
    print('************************')

B    12389
A    10182
C     8740
D     6015
E     3394
F     1300
G      512
Name: grade, dtype: int64
************************
B3    2997
A4    2905
B5    2807
A5    2793
B4    2590
C1    2264
C2    2157
B2    2113
B1    1882
A3    1822
C3    1658
A2    1520
D2    1485
C4    1370
D3    1322
C5    1291
A1    1142
D4    1139
D1    1053
D5    1016
E1     884
E2     791
E3     668
E4     552
E5     499
F1     392
F2     308
F3     236
F4     211
F5     153
G1     141
G2     107
G4      99
G5      86
G3      79
Name: sub_grade, dtype: int64
************************
10+ years                                                                                                                                                                                                                                                                                                                                       9369
< 1 year                                                                                   

RecursionError: ignored

In [58]:
# drop columns that either do no add informaiton or are documeted as part of the label 'loan_status'
dropCatColumn = ['settlement_date','settlement_status','debt_settlement_flag','debt_settlement_flag_date','hardship_flag','application_type','next_pymnt_d','last_pymnt_d','initial_list_status','zip_code', 'addr_state', 'title',
                 'desc','pymnt_plan','issue_d','term','int_rate']

In [59]:
df.drop(dropCatColumn,inplace=True,axis=1)

In [60]:
for floats in df.columns[df.dtypes == 'float64']:
    print(df[floats].value_counts())
    print('************************')

10000.0    3016
12000.0    2439
5000.0     2260
6000.0     2037
15000.0    2012
           ... 
10350.0       1
19100.0       1
17975.0       1
31150.0       1
20425.0       1
Name: loan_amnt, Length: 898, dtype: int64
************************
10000.0    2924
12000.0    2347
5000.0     2247
6000.0     2023
15000.0    1897
           ... 
23625.0       1
19625.0       1
21425.0       1
27750.0       1
850.0         1
Name: funded_amnt, Length: 1051, dtype: int64
************************
5000.000000     1369
10000.000000    1302
6000.000000     1241
12000.000000    1084
8000.000000      929
                ... 
5480.733306        1
14276.431140       1
10141.459840       1
10019.969440       1
1099.996405        1
Name: funded_amnt_inv, Length: 9244, dtype: int64
************************
311.11    68
180.96    59
311.02    54
150.80    48
368.45    46
          ..
357.77     1
809.53     1
48.93      1
405.46     1
204.84     1
Name: installment, Length: 16459, dtype: int64
*************

In [61]:
# Future date/post approval, missing too much data to extrapolate
dropFloats = ['settlement_term','settlement_percentage','settlement_amount','delinq_amnt','acc_now_delinq','policy_code','chargeoff_within_12_mths','collections_12_mths_ex_med','last_pymnt_amnt','collection_recovery_fee','total_rec_late_fee','total_rec_int',
              'total_rec_prncp','total_pymnt_inv','total_pymnt','out_prncp_inv','out_prncp','mths_since_last_record','installment','funded_amnt_inv','funded_amnt','loan_amnt']

In [62]:
df.drop(dropFloats,inplace=True,axis=1)

In [63]:
df.shape

(42538, 23)

In [64]:
df.isna().sum()

grade                         3
sub_grade                     3
emp_title                  2629
emp_length                 1115
home_ownership                3
annual_inc                    7
verification_status           3
loan_status                   3
purpose                       3
dti                           3
delinq_2yrs                  32
earliest_cr_line             32
inq_last_6mths               32
mths_since_last_delinq    26929
open_acc                     32
pub_rec                      32
revol_bal                     3
revol_util                   93
total_acc                    32
recoveries                    3
last_credit_pull_d            7
pub_rec_bankruptcies       1368
tax_liens                   108
dtype: int64

In [65]:
# target value, we cannot use observations missing this value.
df[df['loan_status'].isnull()]

,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,recoveries,last_credit_pull_d,pub_rec_bankruptcies,tax_liens
18286,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18287,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
# This reolves the missing target value, missing grade, and missing dti values
df.drop([18286,18287,18288], inplace=True)

In [67]:
# It is reasonable to assume that no entry is likely a lack of occurance for these fields, although it could be a data entry error. It is unlikey the lendor did not look for these on the credit history.
zero_out = ['tax_liens','pub_rec_bankruptcies','recoveries','mths_since_last_delinq','delinq_2yrs','inq_last_6mths','pub_rec']

for zOut in zero_out:
  df[zOut].fillna(0.0,inplace=True)

In [68]:
df.emp_title.value_counts()

US Army                             139
Bank of America                     115
IBM                                  72
AT&T                                 61
Kaiser Permanente                    61
                                   ... 
Regional Elite Airlines Services      1
Mass General Medical Group            1
Kontera                               1
Southeast Georgia Health ystem        1
Homemaker                             1
Name: emp_title, Length: 30656, dtype: int64

In [69]:
# emp_title is too diverse to be of use. Possible categorization, however 30k+ values makes that impraticle at this time.
df.drop('emp_title',axis=1,inplace=True)

In [70]:
df.home_ownership.value_counts()

RENT        20180
MORTGAGE    18957
OWN          3251
OTHER         136
NONE            8
Name: home_ownership, dtype: int64

In [71]:
# NONE is ambiguous, OTHER category better fits homeless and those living with others.
df['home_ownership'].replace('NONE','OTHER',inplace=True)

In [72]:
df['home_ownership'].fillna('OTHER')
df.home_ownership.value_counts()

RENT        20180
MORTGAGE    18957
OWN          3251
OTHER         144
Name: home_ownership, dtype: int64

In [73]:
df.purpose.value_counts()

debt_consolidation    19775
credit_card            5477
other                  4425
home_improvement       3199
major_purchase         2310
small_business         1991
car                    1615
wedding                1004
medical                 753
moving                  629
house                   426
educational             422
vacation                400
renewable_energy        106
Name: purpose, dtype: int64

In [74]:
# Replace empty fields with other category
purposeNULL = df.purpose.isna()
df = df.replace(purposeNULL,'other')

In [75]:
df.isna().sum()

grade                        0
sub_grade                    0
emp_length                1112
home_ownership               0
annual_inc                   4
verification_status          0
loan_status                  0
purpose                      0
dti                          0
delinq_2yrs                  0
earliest_cr_line            29
inq_last_6mths               0
mths_since_last_delinq       0
open_acc                    29
pub_rec                      0
revol_bal                    0
revol_util                  90
total_acc                   29
recoveries                   0
last_credit_pull_d           4
pub_rec_bankruptcies         0
tax_liens                    0
dtype: int64

In [76]:
df.last_credit_pull_d.value_counts()

Jun-19    8445
Oct-16    3860
May-19    1073
Apr-19     819
Mar-19     700
          ... 
Dec-07       2
Feb-08       2
Jul-08       1
Jun-08       1
Jul-07       1
Name: last_credit_pull_d, Length: 143, dtype: int64

In [77]:
# these columns will take the median value for fillna
median_fill = ['emp_length','annual_inc','open_acc','pub_rec','open_acc','revol_util','total_acc']
for med in median_fill:
  df[med].fillna(df[med].median,inplace=True)

earliest_cr_line and last_credit_pull_d are date based and will be removed for now. They would be of greater use as categories for time periods of age of accounts.

In [78]:
dates = ['earliest_cr_line','last_credit_pull_d']
df.drop(dates,axis=1,inplace=True)

In [79]:
df.isna().sum()

grade                     0
sub_grade                 0
emp_length                0
home_ownership            0
annual_inc                0
verification_status       0
loan_status               0
purpose                   0
dti                       0
delinq_2yrs               0
inq_last_6mths            0
mths_since_last_delinq    0
open_acc                  0
pub_rec                   0
revol_bal                 0
revol_util                0
total_acc                 0
recoveries                0
pub_rec_bankruptcies      0
tax_liens                 0
dtype: int64

In [80]:
df.reset_index(inplace=True) 

In [81]:
pd.options.display.max_columns = None

## Data Exploration

In [91]:
create_report(df).show_browser()

RecursionError: ignored